In [116]:
import pandas as pd

### Read files 

In [265]:
df_train=pd.read_csv(r'C:\San\Projects\Python\salesForecast\data\train.csv\train.csv')
df_train['date'] = pd.to_datetime(df_train['date'])
df_transactions=pd.read_csv(r'C:\San\Projects\Python\salesForecast\data\transactions.csv\transactions.csv')
df_transactions['date'] = pd.to_datetime(df_transactions['date'])
df_holidays=pd.read_csv(r'C:\San\Projects\Python\salesForecast\data\holidays_events.csv')
df_holidays['date'] = pd.to_datetime(df_holidays['date'])
df_oil=pd.read_csv(r'C:\San\Projects\Python\salesForecast\data\oil.csv')
df_oil['date'] = pd.to_datetime(df_oil['date'])

In [157]:
# basic date time values extracted
df_train['dom'] = df_train['date'].dt.day
df_train['dow'] = df_train['date'].dt.day_name()
# https://stackoverflow.com/questions/25249033/week-of-a-month-pandas
df_train['wom'] = df_train['date'].apply(lambda d: (d.day-1) // 7 + 1)
df_train['month'] = df_train['date'].dt.month
df_train['year'] = df_train['date'].dt.year

In [158]:
df_train['isMonthStart']=df_train['date'].dt.is_month_start
df_train['isMonthEnd']=df_train['date'].dt.is_month_end
# https://www.analyticsvidhya.com/blog/2020/03/what-are-lambda-functions-in-python/
# https://stackoverflow.com/questions/44991438/lambda-including-if-elif-else
df_train["wagePaid"] = df_train[["dom", "isMonthEnd"]].apply(lambda x: True if (x.dom == 15) else(True if (x.isMonthEnd == True) else False),axis=1)

In [179]:
df_train['daysFromLastStipend']=df_train['dom'].apply(lambda x: x if (x<15) else (x-15))

df=df_train.merge(df_transactions,on=['date','store_nbr'],how='left').fillna(0)

df_holidays=df_holidays[df_holidays['transferred']!=True].reset_index(drop=True)
df_holidays['type']=True

df_holidays

In [189]:
df=df.merge(df_holidays[['date','type']],on=['date'],how='left',suffixes="isHoliday_").fillna(False)

C:\Users\casap\AppData\Local\Temp\ipykernel_6196\3400281737.py:1: FutureWarning: Passing 'suffixes' as a <class 'str'>, is not supported and may give unexpected results. Provide 'suffixes' as a tuple instead. In the future a 'TypeError' will be raised.
  df=df.merge(df_holidays[['date','type']],on=['date'],how='left',suffixes="isHoliday_").fillna(False)


In [192]:
df.columns = ['id', 'date', 'store_nbr', 'family','sales','onpromotion','dom','dow','wom','month','year','isMonthStart','isMonthEnd','wagePaid','daysFromLastStipend','transactions','isHoliday']

In [267]:
r = pd.date_range(start=df_oil.date.min(), end=df_oil.date.max())
df_oil=df_oil.set_index("date").reindex(r).ffill().bfill().reset_index()
df_oil.columns=['date','oil_price']
df=df.merge(df_oil,on=['date'],how='left').fillna(0)

In [298]:
df.to_csv('finalDataframe.csv',index=False)  

In [297]:
df

,id,date,store_nbr,family,sales,onpromotion,dom,dow,wom,month,year,isMonthStart,isMonthEnd,wagePaid,daysFromLastStipend,transactions,isHoliday,oil_price
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,1,Tuesday,1,1,2013,True,False,False,1,0.0,True,93.14
1,1,2013-01-01,1,BABY CARE,0.000,0,1,Tuesday,1,1,2013,True,False,False,1,0.0,True,93.14
2,2,2013-01-01,1,BEAUTY,0.000,0,1,Tuesday,1,1,2013,True,False,False,1,0.0,True,93.14
3,3,2013-01-01,1,BEVERAGES,0.000,0,1,Tuesday,1,1,2013,True,False,False,1,0.0,True,93.14
4,4,2013-01-01,1,BOOKS,0.000,0,1,Tuesday,1,1,2013,True,False,False,1,0.0,True,93.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,15,Tuesday,3,8,2017,False,False,True,0,2155.0,True,47.57
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,15,Tuesday,3,8,2017,False,False,True,0,2155.0,True,47.57
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,15,Tuesday,3,8,2017,False,False,True,0,2155.0,True,47.57
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,15,Tuesday,3,8,2017,False,False,True,0,2155.0,True,47.57


In [299]:
df_test=pd.read_csv(r'finalDataframe.csv')

In [300]:
df_test

,id,date,store_nbr,family,sales,onpromotion,dom,dow,wom,month,year,isMonthStart,isMonthEnd,wagePaid,daysFromLastStipend,transactions,isHoliday,oil_price
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,1,Tuesday,1,1,2013,True,False,False,1,0.0,True,93.14
1,1,2013-01-01,1,BABY CARE,0.000,0,1,Tuesday,1,1,2013,True,False,False,1,0.0,True,93.14
2,2,2013-01-01,1,BEAUTY,0.000,0,1,Tuesday,1,1,2013,True,False,False,1,0.0,True,93.14
3,3,2013-01-01,1,BEVERAGES,0.000,0,1,Tuesday,1,1,2013,True,False,False,1,0.0,True,93.14
4,4,2013-01-01,1,BOOKS,0.000,0,1,Tuesday,1,1,2013,True,False,False,1,0.0,True,93.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,15,Tuesday,3,8,2017,False,False,True,0,2155.0,True,47.57
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,15,Tuesday,3,8,2017,False,False,True,0,2155.0,True,47.57
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,15,Tuesday,3,8,2017,False,False,True,0,2155.0,True,47.57
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,15,Tuesday,3,8,2017,False,False,True,0,2155.0,True,47.57
